In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-06 04:11:35--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   628KB/s    in 1.7s    

2024-02-06 04:11:38 (628 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [11]:
chars = sorted(list(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [12]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode  = lambda l: ''.join([itos[i] for i in l])

print(encode("Hi there"))
print(decode(encode("Hi there")))

[294109, 687462, 209891, 1042098, 641925, 561489, 925393, 561489]
Hi there


In [13]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([ 288642,  687462,  925393,  975089, 1042098,  209891,  278715,  687462,
        1042098,  687462, 1115393,  561489,  799289,  258225,   39999,  274895,
         561489,  577259,  865087,  925393,  561489,  209891, 1094060,  561489,
         209891,  875895,  925393,  865087,  435520,  561489,  561489,  466878,
         209891,  408576,  799289, 1115037,  209891,  577259, 1068682,  925393,
        1042098,  641925,  561489,  925393,  238100,  209891,  641925,  561489,
         408576,  925393,  209891,  750760,  561489,  209891,  975089,  875895,
         561489,  408576,  695178,  247882,   39999,   39999,  272134,  728517,
         728517,  258225,   39999,  336385,  875895,  561489,  408576,  695178,
         238100,  209891,  975089,  875895,  561489,  408576,  695178,  247882,
          39999,   39999,  288642,  687462,  925393,  975089, 1042098,  209891,
         278715,  687462, 1042098,  687462, 1115393,  561489,  799289,  258225,
      

In [14]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size = 8
train_data[:block_size+1]

tensor([ 288642,  687462,  925393,  975089, 1042098,  209891,  278715,  687462,
        1042098])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([288642]) the target: 687462
When input is tensor([288642, 687462]) the target: 925393
When input is tensor([288642, 687462, 925393]) the target: 975089
When input is tensor([288642, 687462, 925393, 975089]) the target: 1042098
When input is tensor([ 288642,  687462,  925393,  975089, 1042098]) the target: 209891
When input is tensor([ 288642,  687462,  925393,  975089, 1042098,  209891]) the target: 278715
When input is tensor([ 288642,  687462,  925393,  975089, 1042098,  209891,  278715]) the target: 687462
When input is tensor([ 288642,  687462,  925393,  975089, 1042098,  209891,  278715,  687462]) the target: 1042098


In [21]:
torch.manual_seed(1337)
batch_size = 2 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([2, 8])
tensor([[ 311721,  561489, 1042098,  218254,  975089,  209891,  641925,  561489],
        [ 577259,  865087,  925393,  209891, 1042098,  641925,  408576, 1042098]])
targets:
torch.Size([2, 8])
tensor([[ 561489, 1042098,  218254,  975089,  209891,  641925,  561489,  408576],
        [ 865087,  925393,  209891, 1042098,  641925,  408576, 1042098,  209891]])
----
when input is [311721] the target: 561489
when input is [311721, 561489] the target: 1042098
when input is [311721, 561489, 1042098] the target: 218254
when input is [311721, 561489, 1042098, 218254] the target: 975089
when input is [311721, 561489, 1042098, 218254, 975089] the target: 209891
when input is [311721, 561489, 1042098, 218254, 975089, 209891] the target: 641925
when input is [311721, 561489, 1042098, 218254, 975089, 209891, 641925] the target: 561489
when input is [311721, 561489, 1042098, 218254, 975089, 209891, 641925, 561489] the target: 408576
when input is [577259] the target: 865087
w

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        # if targets is None:
        #     loss = None
        # else:
        #     B, T, C = logits.shape
        #     logits = logits.view(B*T, C)
        #     targets = targets.view(B*T)
        #     loss = F.cross_entropy(logits, targets)

        return logits

    # def generate(self, idx, max_new_tokens):
    #     # idx is (B, T) array of indices in the current context
    #     for _ in range(max_new_tokens):
    #         # get the predictions
    #         logits, loss = self(idx)
    #         # focus only on the last time step
    #         logits = logits[:, -1, :] # becomes (B, C)
    #         # apply softmax to get probabilities
    #         probs = F.softmax(logits, dim=-1) # (B, C)
    #         # sample from the distribution
    #         idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
    #         # append sampled index to the running sequence
    #         idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    #     return idx

m = BigramLanguageModel(vocab_size)
# logits, loss = m(xb, yb)
out = m(xb, yb)
print(out.shape)
# print(loss)

# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


RuntimeError: [enforce fail at alloc_cpu.cpp:83] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 4976415100944 bytes. Error code 12 (Cannot allocate memory)